# Get BOM daily data for the weather stations we want to join

Requirements:

cleansed_crashdata_stations.csv

all_metadata.csv (from Case Study 3)

Outputs: 

crashdata_bom_station_data.csv


In [2]:
import numpy as np
import pandas as pd
from scipy import spatial
from scipy.spatial import KDTree
import math
from datetime import datetime, timedelta

In [ ]:
# Load crash data
cleansed_crashdata_stations = pd.read_csv("cleansed_crashdata_stations.csv", low_memory=False)
#cleansed_crashdata_stations.head()

station_locations = pd.read_csv("all_metadata.csv")
station_locations = station_locations.rename(columns={'latitude': 'bom_station_latitude',
                                 'longitude': 'bom_station_longitude',
                                 'name': 'bom_station_name',
                                 'elevation': 'bom_station_elevation',
                                 'filename' : 'bom_station_filename',
                                 'id': 'bom_station_id'})

In [ ]:
# Use code from case study 3 to get weather data for the stations that are relevant to this data set

relevant_bom_station_ids = pd.DataFrame()
relevant_bom_station_ids['bom_station_id'] = cleansed_crashdata_stations['bom_station_id'].unique().tolist()
relevant_bom_station_ids
relevant_bom_station_ids.to_csv("relevant_bom_station_ids")

source_names = {0: 'station',
               13: 'deaccum-nearby',
               15: 'deaccum-interp',
               23: 'nearby-BoM',
               25: 'interp-daily',
               26: 'synth-pan',
               35: 'interp-daily-CLIMARC',
               75: 'interp-lta'}

start_date = 20010101
end_date = 20180630

total_station_data = pd.DataFrame()
#station_list = relevant_bom_station_ids.iloc[1:2]
station_list = relevant_bom_station_ids


def load_data(choice):
    url = 'https://stluc.manta.uqcloud.net/mdatascience/public/datasets/SILO_PPD/' + choice['bom_station_filename']
    print ("Loading {}".format(url))
    data = pd.read_csv(url, header=47, skiprows=[48], skip_blank_lines=False, 
                       delim_whitespace=True, parse_dates=[2], dayfirst=True)
    data = data[(data.Date >= start_date) &  (data.Date <= end_date)]
    return data

#choice = station_locations[station_locations.bom_station_id == 40244].iloc[0]
#data = load_data(choice)

# Gather data for all of the stations we want to assess
for row in station_list.iterrows():
    data = load_data(station_locations[station_locations.bom_station_id == row[1][0]].iloc[0])
    data = data[['Date', 'Day', 'Date2', 'T.Max', 'T.Min', 'Smx', 'Rain', 'Srn']]
    data['station_id'] = row[1][0]
    total_station_data = pd.concat([data, total_station_data])

total_station_data.to_csv("crashdata_bom_station_data.csv", index=False)    


# Link up the data

In [23]:
# Reload the station data if the work book is being restarted at this point
total_station_data = pd.read_csv("crashdata_bom_station_data.csv")
cleansed_crashdata_stations = pd.read_csv("cleansed_crashdata_stations.csv", low_memory=False)

#total_station_data.head()
#cleansed_crashdata_stations.head()

In [53]:
total_station_data_temperature = total_station_data[['Date2', 'T.Max', 'Smx', 'station_id']].copy()
total_station_data_rain = total_station_data[['Date2', 'Rain', 'Srn', 'station_id']].copy()


def subtract_day(date):
    return (pd.to_datetime(date, format = '%Y-%m-%d') - timedelta(1)).strftime('%Y-%m-%d')

#shift dates for rain data 1 day earlier (because they measure rain from 9am the day before to current day)
total_station_data_rain['previous_date'] = total_station_data_rain.apply(lambda x: subtract_day(x['Date2']),axis = 1)

In [54]:
# Check column names
print(total_station_data.columns.tolist())
print(total_station_data_temperature.columns.tolist())
print(total_station_data_rain.columns.tolist())
print(cleansed_crashdata_stations.columns.tolist())

['Date', 'Day', 'Date2', 'T.Max', 'Smx', 'Smn', 'Rain', 'Srn', 'station_id']
['Date2', 'T.Max', 'Smx', 'station_id']
['Date2', 'Rain', 'Srn', 'station_id', 'previous_date']
['Crash_Ref_Number', 'Crash_Year', 'Crash_Month', 'Crash_Day_Of_Week', 'Crash_Hour', 'imputed_date', 'DateImputionAgreement', 'month_group', 'Crash_Nature', 'Crash_Type', 'Crash_Longitude_GDA94', 'Crash_Latitude_GDA94', 'Crash_Street', 'Crash_Street_Intersecting', 'Loc_Suburb', 'Loc_Local_Government_Area', 'Loc_Post_Code', 'Loc_Police_Division', 'Loc_Police_District', 'Loc_Police_Region', 'Loc_Queensland_Transport_Region', 'Crash_Roadway_Feature', 'Crash_Traffic_Control', 'Crash_Speed_Limit', 'Crash_Road_Surface_Condition', 'Crash_Atmospheric_Condition', 'Crash_Lighting_Condition', 'Count_Casualty_Fatality', 'Count_Casualty_Hospitalised', 'Count_Casualty_MedicallyTreated', 'Count_Casualty_MinorInjury', 'Count_Casualty_Total', 'Count_Unit_Car', 'Count_Unit_Motorcycle_Moped', 'Count_Unit_Truck', 'Count_Unit_Bus', 'Cou

In [55]:
print(type(total_station_data_temperature['Date2'][10]))
print(total_station_data_temperature['Date2'][10])
print(type(total_station_data_rain['previous_date'][10]))
print(total_station_data_rain['previous_date'][10])

<class 'str'>
2001-01-11
<class 'str'>
2001-01-10


In [59]:
# Merge with temperature data
crashdata_with_daily_weather = pd.merge(cleansed_crashdata_stations, total_station_data_temperature, how='left', left_on = ['nearest_bom_st_id', 'imputed_date'], right_on = ['station_id', 'Date2' ])

#Drop 'Date2' and 'station_id' columns as it they are unnecessary
crashdata_with_daily_weather.drop(['Date2', 'station_id'], axis = 1, inplace = True)

# Merge with weather data (until 9am)
crashdata_with_daily_weather = pd.merge(crashdata_with_daily_weather, total_station_data_rain, how='left', left_on = ['nearest_bom_st_id', 'imputed_date'], right_on = ['station_id', 'Date2' ])

#Drop 'Date2' and 'station_id' columns as it they are unnecessary
crashdata_with_daily_weather.drop(['Date2', 'station_id', 'Srn', 'previous_date'], axis = 1, inplace = True)
crashdata_with_daily_weather.rename(columns = {'Rain': 'Rain until 9am'}, inplace = True)

# Merge with weather data (from 9am)
crashdata_with_daily_weather = pd.merge(crashdata_with_daily_weather, total_station_data_rain, how='left', left_on = ['nearest_bom_st_id', 'imputed_date'], right_on = ['station_id', 'previous_date' ])

#Drop 'Date2' and 'station_id' columns as it they are unnecessary
crashdata_with_daily_weather.drop(['Date2', 'station_id', 'previous_date'], axis = 1, inplace = True)
crashdata_with_daily_weather.rename(columns = {'Rain': 'Rain from 9am'}, inplace = True)


In [61]:
crashdata_with_daily_weather.to_csv("crashdata_with_daily_weather.csv", index=False)
crashdata_with_daily_weather.head(30)



,Crash_Ref_Number,Crash_Year,Crash_Month,Crash_Day_Of_Week,Crash_Hour,imputed_date,DateImputionAgreement,month_group,Crash_Nature,Crash_Type,...,bom_station_elevation,bom_station_id,bom_station_longitude,bom_station_name,nearest_bom_st_distance_km,T.Max,Smx,Rain until 9am,Rain from 9am,Srn
0,1.0,2001,January,Monday,6,NaN,False,1,Head-on,Multi-Vehicle,...,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,1.336785,NaN,NaN,NaN,NaN,NaN
1,2.0,2001,January,Wednesday,9,NaN,False,1,Angle,Multi-Vehicle,...,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,2.344483,NaN,NaN,NaN,NaN,NaN
2,3.0,2001,January,Thursday,8,NaN,False,1,Rear-end,Multi-Vehicle,...,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,2.639759,NaN,NaN,NaN,NaN,NaN
3,4.0,2001,January,Sunday,8,NaN,False,1,Hit object,Single Vehicle,...,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,3.423516,NaN,NaN,NaN,NaN,NaN
4,6.0,2001,January,Wednesday,9,NaN,False,1,Angle,Multi-Vehicle,...,12.0,40211.0,153.0078,ARCHERFIELD AIRPORT,0.898300,NaN,NaN,NaN,NaN,NaN
5,7.0,2001,January,Wednesday,17,NaN,False,1,Hit object,Single Vehicle,...,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,1.382132,NaN,NaN,NaN,NaN,NaN
6,8.0,2001,January,Thursday,9,NaN,False,1,Hit pedestrian,Hit pedestrian,...,12.0,40211.0,153.0078,ARCHERFIELD AIRPORT,1.513006,NaN,NaN,NaN,NaN,NaN
7,9.0,2001,January,Sunday,15,NaN,False,1,Angle,Multi-Vehicle,...,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,3.114004,NaN,NaN,NaN,NaN,NaN
8,10.0,2001,January,Tuesday,15,NaN,False,1,Angle,Multi-Vehicle,...,12.0,40211.0,153.0078,ARCHERFIELD AIRPORT,1.283760,NaN,NaN,NaN,NaN,NaN
9,11.0,2001,January,Wednesday,19,NaN,False,1,Hit object,Single Vehicle,...,12.0,40211.0,153.0078,ARCHERFIELD AIRPORT,1.176132,NaN,NaN,NaN,NaN,NaN


## Validate joined weather

In [3]:
# Read data again in case starting part way through
crashdata_with_daily_weather = pd.read_csv("crashdata_with_daily_weather.csv")

In [29]:
# Get info on data not missing daily weather
not_missing_daily_weather = crashdata_with_daily_weather[~np.isnan(crashdata_with_daily_weather['Srn'])]
print("Max date with daily weather")
print(not_missing_daily_weather[not_missing_daily_weather['imputed_date'] == not_missing_daily_weather['imputed_date'].max()].iloc[0].imputed_date)
print("Min date with daily weather")
print(not_missing_daily_weather[not_missing_daily_weather['imputed_date'] == not_missing_daily_weather['imputed_date'].min()].iloc[0].imputed_date)

print("")
print("Summary atmospheric conditions (not missing daily data)")
print(not_missing_daily_weather['Crash_Atmospheric_Condition'].value_counts())

print("")
print("Summary atmospheric conditions (all)")
print(crashdata_with_daily_weather['Crash_Atmospheric_Condition'].value_counts())

Max date with daily weather
2017-02-10
Min date with daily weather
2001-01-01

Summary atmospheric conditions (not missing daily data)
Clear         148949
Raining        20507
Fog              567
Unknown          143
Smoke/Dust       135
Name: Crash_Atmospheric_Condition, dtype: int64

Summary atmospheric conditions (all)
Clear         212411
Raining        28835
Fog              783
Smoke/Dust       216
Unknown          177
Name: Crash_Atmospheric_Condition, dtype: int64


In [42]:
def validate_rain(crashdata):
    rainfall_disagreement = crashdata[( \
    (crashdata['Crash_Atmospheric_Condition'] == 'Raining') & \
    ( ( (crashdata['Crash_Hour'] <= 9) & (crashdata['Rain until 9am'] == 0))  | \
     ( (crashdata['Crash_Hour'] > 9) & (crashdata['Rain from 9am'] == 0))))
    ]
    rainfall_agreement =  pd.concat([crashdata, rainfall_disagreement]).drop_duplicates(keep = False)
    outputmsg = ""
    # Check sets add up
    outputmsg += "Total crashes with daily data: \n"
    outputmsg += str(crashdata.shape[0])
    outputmsg += ("\nTotal crashes with disagreement on rainfall\n")
    outputmsg += str(rainfall_disagreement.shape[0])
    outputmsg += ("\nTotal crashes with agreement on rainfall\n")
    outputmsg += str(rainfall_agreement.shape[0])
    outputmsg += ("\nSum of agreement and disagreement\n")
    outputmsg += str(rainfall_disagreement.shape[0] + rainfall_agreement.shape[0])
    outputmsg += ("\nPercentage of disagreement\n")
    outputmsg += str((rainfall_disagreement.shape[0] / crashdata.shape[0]) * 100)

    # Compare average distance to mean
    outputmsg += ("\n\nAverage distance to station\n")
    outputmsg += str(crashdata['nearest_bom_st_distance_km'].mean())
    outputmsg += ("\nAverage distance to station when rainfall disagrees\n")
    outputmsg += str(rainfall_disagreement['nearest_bom_st_distance_km'].mean())
    outputmsg += ("\nAverage distance to station when rainfall agrees\n")
    outputmsg += str(rainfall_agreement['nearest_bom_st_distance_km'].mean())
    return(crashdata, rainfall_disagreement, rainfall_agreement, outputmsg)

daily_weather_rain_disagreement = validate_rain(not_missing_daily_weather)
daily_weather_1km_rain_disagreement = validate_rain(crashdata_with_daily_weather[crashdata_with_daily_weather['nearest_bom_st_distance_km'] <1])

print("Summary of rain disagreement for all crashes with daily data:")
print(daily_weather_rain_disagreement[3])

print("\nSummary of rain disagreement for all crashes within 1km of station with daily data:")
print(daily_weather_1km_rain_disagreement[3])


Summary of rain disagreement for all crashes with daily data:
Total crashes with daily data: 
170301
Total crashes with disagreement on rainfall
1152
Total crashes with agreement on rainfall
169149
Sum of agreement and disagreement
170301
Percentage of disagreement
0.6764493455704899

Average distance to station
3.4664387496080566
Average distance to station when rainfall disagrees
3.937447022569446
Average distance to station when rainfall agrees
3.4632309178712255

Summary of rain disagreement for all crashes within 1km of station with daily data:
Total crashes with daily data: 
27350
Total crashes with disagreement on rainfall
115
Total crashes with agreement on rainfall
27235
Sum of agreement and disagreement
27350
Percentage of disagreement
0.4204753199268739

Average distance to station
0.6252326051553941
Average distance to station when rainfall disagrees
0.6339876173913046
Average distance to station when rainfall agrees
0.6251956370479181


In [44]:
# show events that disagreee
daily_weather_rain_disagreement[1]

,Unnamed: 0,Crash_Ref_Number,Crash_Year,Crash_Month,Crash_Day_Of_Week,Crash_Hour,imputed_date,DateImputionAgreement,month_group,Crash_Nature,...,bom_station_elevation,bom_station_id,bom_station_longitude,bom_station_name,nearest_bom_st_distance_km,T.Max,Smx,Rain until 9am,Rain from 9am,Srn
110,110,112.0,2001,June,Wednesday,15,2001-06-20,True,6,Hit object,...,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,3.970092,23.5,25.0,0.0,0.0,0.0
602,602,605.0,2003,November,Monday,14,2003-11-10,True,35,Rear-end,...,12.0,40211.0,153.0078,ARCHERFIELD AIRPORT,1.748247,26.0,0.0,1.0,0.0,0.0
803,803,806.0,2004,October,Saturday,7,2004-10-30,True,46,Hit object,...,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,3.308600,23.5,25.0,0.0,0.0,0.0
1438,1438,1441.0,2007,October,Wednesday,23,2007-10-03,True,82,Hit object,...,28.0,40463.0,152.9875,OXLEY,1.774175,31.5,25.0,0.0,0.0,0.0
1758,1758,1761.0,2009,March,Friday,9,2009-03-20,True,99,Angle,...,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,1.213138,29.0,25.0,0.0,1.4,0.0
1889,1889,1892.0,2009,October,Friday,9,2009-10-30,True,106,Rear-end,...,60.0,40244.0,153.0583,SUNNYBANK BOWLS CLUB,3.750554,26.5,25.0,0.0,0.0,0.0
3400,3400,3486.0,2004,March,Tuesday,15,2004-03-16,True,249,Sideswipe,...,15.0,40215.0,153.0333,BRISBANE BOTANICAL GARDENS,2.019594,30.0,25.0,0.0,0.0,25.0
3452,3452,3550.0,2004,July,Monday,20,2004-07-19,True,253,Hit object,...,6.0,40220.0,153.0564,COORPAROO BOWLS CLUB,3.129874,20.5,25.0,0.0,0.0,25.0
3586,3586,3715.0,2005,September,Wednesday,12,2005-09-14,True,267,Angle,...,6.0,40220.0,153.0564,COORPAROO BOWLS CLUB,2.702349,25.0,25.0,0.0,0.0,25.0
4304,4304,4637.0,2015,March,Wednesday,9,2015-03-18,True,380,Hit object,...,6.0,40220.0,153.0564,COORPAROO BOWLS CLUB,2.206392,28.5,25.0,0.0,3.2,25.0
